In [1]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation, Reshape, Input
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras import backend as K

% matplotlib inline

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import h5py

Using TensorFlow backend.


In [2]:
data = h5py.File('fonts.hdf5')
fonts = data['fonts']

In [3]:
%%time

image_list = np.array(fonts).reshape(56443*62, 64, 64, 1)
print(image_list.shape)

(3499466, 64, 64, 1)
CPU times: user 9.36 s, sys: 17.9 s, total: 27.2 s
Wall time: 2min 42s


In [4]:
all_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
chars = { i: char for i, char in enumerate(all_chars) }

In [14]:
X = image_list
y = np.array([i for _ in range(56443) for i in range(62)])

print(X.shape)
print(y.shape)

(3499466, 64, 64, 1)
(3499466,)


In [18]:
# Be careful which server you're on when running this one. It will generate about 6.2G of data.

def create_dataset(X, y, start, interval=100000, save=False):
    X_subset = X[start:start+interval]
    y_subset = y[start:start+interval]
    
    if save:
        np.savez_compressed("np_arrays/subset_{}-{}.npz".format(str(start), str(start+interval)), X=X_subset, y=y_subset)
    
    print("X shape: {}".format(X_subset.shape))
    print("y shape: {}".format(y_subset.shape))
    print("X dtype: {}".format(X_subset.dtype))
    print("y dtype: {}".format(y_subset.dtype))
    
    return X_subset, y_subset

#X_subset, y_subset = create_dataset(X, y, 300000, save=True)
#X_subset, y_subset = create_dataset(X, y, 400000, save=True)

X shape: (100000, 64, 64, 1)
y shape: (100000,)
X dtype: uint8
y dtype: int64


In [16]:
%%time

X_subset = X_subset / 255
y_subset = to_categorical(y_subset)

print(X_subset.shape)
print(y_subset.shape)

print(X_subset.dtype)
print(y_subset.dtype)

(100000, 64, 64, 1)
(100000, 62)
float64
float64
CPU times: user 2.77 s, sys: 1.03 s, total: 3.8 s
Wall time: 3.8 s


In [8]:
X_subset_train, X_subset_test, y_subset_train, y_subset_test = train_test_split(X_subset, 
                                                                                y_subset, 
                                                                                test_size = .25, 
                                                                                random_state = 42
                                                                               )
print(X_subset_train.shape)
print(X_subset_test.shape)
print(y_subset_train.shape)
print(y_subset_test.shape)

(75000, 64, 64, 1)
(25000, 64, 64, 1)
(75000, 62)
(25000, 62)


In [9]:
src_model = 'font_model_second100000samples_8epochs.h5'

In [73]:
model = load_model(src_model)

In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 256)       1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 512)       524800    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 512)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 512)       2048      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 1024)      2098176   
__________

In [75]:
model.layers.pop()
model.layers.pop()
model.layers.pop()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 256)       1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 512)       524800    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 512)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 512)       2048      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 1024)      2098176   
__________

In [76]:
# Freeze original layers

for layer in model.layers:
    layer.trainable = False

In [77]:
model.add(Dense(512, activation='relu'))

#model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
#model.add(MaxPooling2D(pool_size=2))
#model.add(BatchNormalization())
#model.add(GlobalAveragePooling2D())
#model.add(Dense(512, activation='relu'))
#model.add(Dense(62, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 256)       1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 512)       524800    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 512)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 512)       2048      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 1024)      2098176   
__________